In [1]:
import pandas as pd

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import LancasterStemmer
stemmer =  LancasterStemmer()
lemmer = WordNetLemmatizer()

In [3]:
from glove import Corpus, Glove
import numpy as np
from pandas import DataFrame, Series

In [18]:
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [19]:
from nltk.corpus import stopwords

stop = stopwords.words('russian')
stop += stopwords.words('english')

my_stop = ['весь', 'всякий', 'тот', 'это', 'самое', 'каким', 'сама', 'никак', 'она', 'она', 'оно', 'какие', 'какого', 'которая', 'многое', 'чему', 'всему', 'раза', 'сразу', 'весь', 'раз', 'пор', 'например', 'вроде', 'которые', 'который', 'просто', 'очень', 'почему', 'вообще', 'ещё', 'типа', 'ради', 'всё', 'хотя', 'тебе', 'нужно', 'пока']
my_stop_verb = ['сделать', 'можешь', 'могу', 'могут', 'делать', 'будешь', 'быть', 'будут', 'смочь']

[stop.append(morph.parse(i)[0].normal_form) for i in my_stop]
[stop.append(morph.parse(i)[0].normal_form) for i in my_stop_verb]

dvach_stop = ['такое', 'лол', 'бля']
stop += dvach_stop

In [ ]:
alpha_tokenizer = RegexpTokenizer('[A-Za-zА-Яа-я]\w+')

def my_tokenize(sent):
    sents = list(alpha_tokenizer.tokenize(sent))
    return list([morph.parse(sent.lower())[0].normal_form for sent in sents if sent.lower() not in stop and len(sent.lower()) > 1])

model_data = [my_tokenize(sent) for sent in open('2ch_corpus.txt').readlines()]

In [33]:
corpus = Corpus()
corpus.fit(model_data, window=3)

In [34]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [35]:
glove.add_dictionary(corpus.dictionary)

In [36]:
g_vocab = glove.dictionary

In [37]:
glove.most_similar('лох', number = 10)

[('дурачок', 0.71623993262774766),
 ('гнать', 0.68904416079437514),
 ('ебать', 0.68828226003396542),
 ('необучаемый', 0.64642688839043183),
 ('стингер', 0.63682430224616948),
 ('имбецил', 0.61763937666303359),
 ('говноед', 0.60460652212664245),
 ('лаж', 0.6030902427839252),
 ('дальневосточник', 0.60230146652025562)]

In [38]:
from os import path, makedirs
from pickle import dump

subdir = 'models'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, 'glove_2ch')

with open(file_path, 'wb') as fp:
    dump(glove, fp)